In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder

from qiskit_metal.qlibrary.passives.cap_three_fingers import CapThreeFingers
from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond

In [4]:
design = metal.designs.DesignPlanar()
gui = metal.MetalGUI(design)

In [5]:
design.chips.main.size['size_x'] = '9mm'
design.chips.main.size['size_y'] = '6mm'

design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [6]:
## Basic operation
options = dict(
    pad_width = '425 um',
    pad_height = '90um',
    pocket_height = '650 um',
    cl_pocket_edge = '180',
    connection_pads = dict(
        readout = dict(loc_W=+1, loc_H=+1),
        bus = dict(loc_W=-1, loc_H=-1)
    ))

Q1 = TransmonPocket(design,'Q1', options = dict(
        pos_x='-1.0mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '14nH',
        **options))

Q2 = TransmonPocket(design,'Q2', options = dict(
        pos_x='+1.0mm',
        pos_y='+0mm',
        gds_cell_name = 'TOP',
        hfss_inductance = '24nH',
        **options))

gui.rebuild()
gui.autoscale()

In [7]:
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.1mm',
            end_straight='0.1mm'),
        trace_gap='9um',
        trace_width='15um')

asym = 150
cpw1 = connect('cpw1', 'Q1', 'readout', 'Q2', 'bus', '5 mm', f'+{asym}um')

gui.rebuild()
gui.autoscale()

In [8]:
Q1.options.pad_height='80um' 
Q2.options.pad_height='80um'